# Using rabbitmq-server Message Broker with Amazon AWS

### Installation

check notes on http://kronosapiens.github.io/blog/2015/04/28/rabbitmq-aws.html

Install rabbitmq-server server first:
sudo apt-get install rabbitmq-server

check if the rabbitmq server is up and runnning: sudo rabbitmqctl status

RabbitMQ comes with a “guest” user out of the box, but this user is not configured to accept external requests. This would be fine if you were planning on sending and consuming messages locally, but we are not.

The easiest way to configure RabbitMQ to accept external requests is to create a new user. Go ahead and run the following. (Feel free to chahnge the myguest username and password to anytihng you like):

sudo rabbitmqctl add_user myguest myguestpwd

sudo rabbitmqctl set_permissions -p / myguest ".*" ".*" ".*"

This will create a new user on the RabbitMQ server. We will use this username and password to configure the app-layer instance to send messages to Rabbit.

Let’s check to make sure the user was created correctly:

sudo rabbitmqctl list_users

In [ ]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [1]:
#!pip install pika

In [3]:
import pika
import sys

### Example for sending messages. They will be consumbed by workers  

In [10]:
def send_message(messagetxt):
    
    credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
    #URL='localhost'
    URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
    parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()
    
    channel.queue_declare(queue='task_queue', durable=True)
    message = ' '.join(messagetxt) or "Hello World!"
    
    channel.basic_publish(exchange='',
                      routing_key='task_queue',
                      body=message,
                      properties=pika.BasicProperties(
                         delivery_mode = 2, # make message persistent
                      ))
    print(" [x] Sent %r" % message)
    connection.close()
    

In [12]:
send_message('First.')
send_message('Second..')
send_message('Third...')
send_message('Fourth....')
send_message('Fifth.....')




 [x] Sent 'F i r s t .'
 [x] Sent 'S e c o n d . .'
 [x] Sent 'T h i r d . . .'
 [x] Sent 'F o u r t h . . . .'
 [x] Sent 'F i f t h . . . . .'
